In [ ]:
from Crawling import Crawling

crawl = Crawling()
BGM_data = crawl.get_BGM_data()

In [ ]:
from Database import Database
DB = Database('alskfl')
DB.update_song(BGM_data)
DB.update_list(BGM_data)

In [ ]:
def update_song_list(BGM_data):
    for list in BGM_data.keys():
        list_id = DB.get_list_id(list)
        crawled_songs = [x[2] for x in BGM_data[list]['items']]
        db = DB.connect()
        cursor = db.cursor()
        sql = f'''
            SELECT song.href FROM song_list
            JOIN song ON song_list.id = song.id
        '''
        cursor.execute(sql)
        exist_songs = [x[0] for x in cursor.fetchall()]
        insert_songs = [x for x in crawled_songs if x not in exist_songs]
        for song_href in insert_songs:
            song_id = DB.get_song_id(song_href)
            sql = f'''
                INSERT INTO song_list
                (song_id, list_id)
                VALUES
                ({song_id}, {list_id});
            '''
            cursor.execute(sql)
        db.commit()
        db.close()
        break

update_song_list(BGM_data)

![](youtube_playlist.png)